In [22]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
import re
import logging

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('langgraph_workflow.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# init model from ollama
model = init_chat_model(
    # "ollama:gemma3:12b-it-qat",
    "ollama:gemma3:4b"
)

logger.info("Model initialized successfully")

2025-07-06 11:05:20,683 - INFO - Model initialized successfully


In [12]:
def get_weather(location: str):
    """Returns weather info for a location.
    
    Args:
        location (str): The location to get the weather for.

    Returns:
        str: A string describing the weather.

    Example:
        >>> get_weather("San Francisco")
    """
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."

# Available tools dictionary
TOOLS = {
    "get_weather": get_weather
}

In [13]:
import inspect

def create_tool_description(tools: dict):
    """Creates a string description of available tools that we can pass to the model.
    
    Includes name, description and usage information for each tool.
    """
    
    # for each tool, return its docstring
    tool_descriptions = []
    for tool_name, tool_func in tools.items():
        signature = inspect.signature(tool_func)
        docstring = tool_func.__doc__
        tool_descriptions.append(f"def {tool_name}{signature}:\n\"\"\"{docstring}\n\"\"\"")
    return "\n".join(tool_descriptions)

# test that it works
tool_description = create_tool_description(TOOLS)
logger.info(f"Tool description created:\n{tool_description}")

2025-07-06 10:56:54,073 - INFO - Tool description created:
def get_weather(location: str):
"""Returns weather info for a location.

    Args:
        location (str): The location to get the weather for.

    Returns:
        str: A string describing the weather.

    Example:
        >>> get_weather("San Francisco")
    
"""


In [37]:
# Ensure you do not include any "." in the prompt - you will get errors during the function call!S
instruction_prompt = f'''
# Instructions
You are a helpful conversational AI assistant.
At each turn, if you decide to invoke any of the function(s), it should be wrapped with ```tool_code```.
The python methods described below are imported and available, you can only use defined methods.
ONLY use the ```tool_code``` format when absolutely necessary to answer the user's question.
The generated code should be readable and efficient. 

For questions that don't require any specific tools, just respond normally without tool calls.

# Instructions for using tools:
- Never use print statements. All tool outputs are automatically handled. Only use the tool call format as shown.
- The response to a method will be wrapped in ```tool_output``` use it to call more tools or generate a helpful, friendly response.
- When using a ```tool_call``` think step by step why and how it should be used. 
- All tools will directly output a string into the `tool_output` variable. 

The following Python methods are available:

```python
{tool_description}
```

# Example usage of tools:
You can use a tool like this:
```tool_code
my_tool("argument1", "argument2")
```
- Where 'my_tool' is the name of the tool you want to call, and 'argument1', 'argument2' are the arguments you want to pass to the tool.

# Bad example of tool usage:
```tool_code
result = my_tool("argument1", "argument2")
print(result)
```
- This code will cause an error because the tool output is not being used correctly.

```tool_code
print(my_tool("argument1", "argument2"))
```
- This code will cause an error because the tool output is not being used correctly.
'''

print(instruction_prompt)


# Instructions
You are a helpful conversational AI assistant.
At each turn, if you decide to invoke any of the function(s), it should be wrapped with ```tool_code```.
The python methods described below are imported and available, you can only use defined methods.
ONLY use the ```tool_code``` format when absolutely necessary to answer the user's question.
The generated code should be readable and efficient. 

For questions that don't require any specific tools, just respond normally without tool calls.

# Instructions for using tools:
- Never use print statements. All tool outputs are automatically handled. Only use the tool call format as shown.
- The response to a method will be wrapped in ```tool_output``` use it to call more tools or generate a helpful, friendly response.
- When using a ```tool_call``` think step by step why and how it should be used. 
- All tools will directly output a string into the `tool_output` variable. 

The following Python methods are available:

```python

In [26]:
def extract_tool_calls(text):
    """Extract tool calls from model output using regex parsing."""
    logger.info(f"[TOOL_PARSER] Starting tool extraction from text: {text[:500]}...")
    
    pattern = r"```tool_code\s*(.*?)\s*```"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        code = match.group(1).strip()
        logger.info(f"[TOOL_PARSER] Found tool code: {code}")
        
        try:
            logger.info(f"[TOOL_PARSER] Attempting to execute: {code}")
            logger.info(f"[TOOL_PARSER] Available tools: {list(TOOLS.keys())}")
            
            # Execute the tool call safely
            result = eval(code, {"__builtins__": {}}, TOOLS)
            logger.info(f"[TOOL_PARSER] Tool execution successful: {result}")
            
            return f'```tool_output\n{result}\n```'
        except Exception as e:
            logger.error(f"[TOOL_PARSER] Tool execution failed: {str(e)}")
            logger.error(f"[TOOL_PARSER] Error type: {type(e).__name__}")
            logger.error(f"[TOOL_PARSER] Code that failed: {code}")
            return f'```tool_output\nError: {str(e)}\n```'
    else:
        logger.info("[TOOL_PARSER] No tool_code blocks found in text")
        return None

In [16]:
def react_agent(state: MessagesState):
    """Single ReAct agent that can generate responses and execute tools in a loop."""
    messages = state["messages"]
    logger.info(f"[REACT] Processing {len(messages)} messages")
    
    # Always include the system prompt for tool instructions
    system_prompt = instruction_prompt
    
    # Build conversation with system prompt
    conversation = [{"role": "system", "content": system_prompt}] + messages
    
    # Generate response
    response = model.invoke(conversation)
    logger.info(f"[REACT] Model response: {response.content[:200]}...")
    
    # Check if response contains tool calls
    if '```tool_code' in str(response.content):
        logger.info("[REACT] Tool code detected - executing tools")
        
        # Execute the tool call
        tool_output = extract_tool_calls(response.content)
        
        if tool_output:
            logger.info(f"[REACT] Tool execution result: {tool_output}")
            
            # Extract the result from tool_output
            result_match = re.search(r'```tool_output\n(.*?)\n```', tool_output, re.DOTALL)
            if result_match:
                clean_result = result_match.group(1).strip()
                logger.info(f"[REACT] Clean result: {clean_result}")
                
                # Create a new response incorporating the tool result
                final_response_prompt = f"""Based on the tool result: {clean_result}
                
Please provide a helpful, natural response to the user incorporating this information. 
Do not include any tool code or technical details, just a conversational answer."""
                
                # Generate final response with tool result
                final_conversation = [
                    {"role": "system", "content": final_response_prompt},
                    {"role": "user", "content": messages[-1].content}
                ]
                
                final_response = model.invoke(final_conversation)
                logger.info(f"[REACT] Final response with tool result: {final_response.content}")
                
                return {"messages": [final_response]}
    
    # No tool calls needed, return the response as-is
    logger.info("[REACT] No tool calls detected - returning response")
    return {"messages": [response]}

def should_continue_react(state: MessagesState):
    """Always end after the react agent processes the input."""
    return "end"

In [17]:
# Simplified ReAct graph setup 
builder = StateGraph(MessagesState)
builder.add_node("react", react_agent)

builder.add_edge(START, "react")
builder.add_edge("react", END)
graph = builder.compile()

In [18]:
# Enhanced utils for better debugging
def print_conversation(result):
    print("=== CONVERSATION FLOW ===")
    messages = result["messages"]
    
    for i, message in enumerate(messages):
        print(f"\n--- Message {i+1} ---")
        print(f"Type: {type(message).__name__}")
        print(f"Content: {message.content}")
        
        # Check if this message contains a tool call
        if '```tool_code' in str(message.content):
            print("🔧 TOOL CALL DETECTED")
            
            # Extract the tool code for debugging
            pattern = r"```tool_code\s*(.*?)\s*```"
            match = re.search(pattern, message.content, re.DOTALL)
            if match:
                code = match.group(1).strip()
                print(f"📝 Tool Code: {code}")
                
                # Try to execute and show result
                tool_output = extract_tool_calls(message.content)
                if tool_output:
                    print(f"🔧 Tool Result: {tool_output}")
                else:
                    print("❌ No tool output generated")
            else:
                print("❌ Could not extract tool code")
        
        # Show if this is a tool output
        if '```tool_output' in str(message.content):
            print("📊 TOOL OUTPUT DETECTED")
            
            # Extract the tool output for debugging
            pattern = r"```tool_output\n(.*?)\n```"
            match = re.search(pattern, message.content, re.DOTALL)
            if match:
                output = match.group(1).strip()
                print(f"📋 Output: {output}")
        
        print("-" * 50)

In [23]:
#excecute graph - test that chat history works
input_prompt = "You are a helpful assistant named Tim"
query = "What is your name? I am barry!"
messages = [
    SystemMessage(content=input_prompt),
    HumanMessage(content=query)
]
state : MessagesState = {"messages": messages}
result = graph.invoke({"messages": state["messages"]})
print_conversation(result)

2025-07-06 11:05:27,611 - INFO - [REACT] Processing 2 messages
2025-07-06 11:05:29,750 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-07-06 11:05:30,060 - INFO - [REACT] Model response: Hi Barry! My name is Tim. It’s nice to meet you!
...
2025-07-06 11:05:30,062 - INFO - [REACT] No tool calls detected - returning response


=== CONVERSATION FLOW ===

--- Message 1 ---
Type: SystemMessage
Content: You are a helpful assistant named Tim
--------------------------------------------------

--- Message 2 ---
Type: HumanMessage
Content: What is your name? I am barry!
--------------------------------------------------

--- Message 3 ---
Type: AIMessage
Content: Hi Barry! My name is Tim. It’s nice to meet you!

--------------------------------------------------


In [41]:
# Test the simplified ReAct graph with weather query
input_prompt = "You are a helpful assistant named Tim"
query = "What is the weather in San Francisco?"
messages = [
    SystemMessage(content=input_prompt),
    HumanMessage(content=query)
]
state : MessagesState = {"messages": messages}
result = graph.invoke({"messages": state["messages"]})
print_conversation(result)

2025-07-06 11:13:30,629 - INFO - [REACT] Processing 2 messages
2025-07-06 11:13:31,454 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-07-06 11:13:31,744 - INFO - [REACT] Model response: ```tool_code
get_weather(location="San Francisco")
```...
2025-07-06 11:13:31,745 - INFO - [REACT] Tool code detected - executing tools
2025-07-06 11:13:31,745 - INFO - [TOOL_PARSER] Starting tool extraction from text: ```tool_code
get_weather(location="San Francisco")
```...
2025-07-06 11:13:31,746 - INFO - [TOOL_PARSER] Found tool code: get_weather(location="San Francisco")
2025-07-06 11:13:31,747 - INFO - [TOOL_PARSER] Attempting to execute: get_weather(location="San Francisco")
2025-07-06 11:13:31,747 - INFO - [TOOL_PARSER] Available tools: ['get_weather']
2025-07-06 11:13:31,748 - INFO - [TOOL_PARSER] Tool execution successful: It's 60 degrees and foggy.
2025-07-06 11:13:31,748 - INFO - [REACT] Tool execution result: ```tool_output
It's 60 degrees and foggy.
```


=== CONVERSATION FLOW ===

--- Message 1 ---
Type: SystemMessage
Content: You are a helpful assistant named Tim
--------------------------------------------------

--- Message 2 ---
Type: HumanMessage
Content: What is the weather in San Francisco?
--------------------------------------------------

--- Message 3 ---
Type: AIMessage
Content: Okay, it’s looking pretty hazy out there – it’s 60 degrees and foggy in San Francisco today. A bit chilly and mysterious, isn’t it?
--------------------------------------------------
